In [1]:
% matplotlib inline

from __future__ import division
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, squareform
import time as tm
import gsw as sw
from scipy.optimize import curve_fit
from scipy.stats import chi2

import numpy.ma as ma

from mpl_toolkits.basemap import Basemap, cm
from netCDF4 import Dataset as NetCDFFile

from matplotlib import rcParams, rc
rcParams.update({'figure.autolayout': True})
rcParams.update({'font.family': 'serif'})
rcParams.update({'font.serif': 'Times New Roman'})
#rc('text', usetex=True)

/Users/sclayton/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
# set data directory 
indir = r'/Users/sclayton/Google Drive/2014 IrnBru/'

trackFile = r'%sCrunched Met/IrnBru MET corr continuous.xlsx' % indir
track = pd.read_excel(trackFile, header = 1)
track.dropna

print track

                  DATE TIME True Wind spd True Wind dir Air Temp  \
0                       GMT         m/sec       Degrees   Deg. C   
1                       NaN           NaN           NaN      NaN   
2       2014-07-03 00:00:06           4.4         292.2    20.07   
3       2014-07-03 00:00:21           4.4         290.8    20.09   
4       2014-07-03 00:00:36           3.6         295.6    20.11   
5       2014-07-03 00:00:51           3.3         298.9    20.14   
6       2014-07-03 00:01:06           3.1         306.8    20.18   
7       2014-07-03 00:01:21           3.2         315.2    20.22   
8       2014-07-03 00:01:36           3.4           317    20.25   
9       2014-07-03 00:01:51             4         312.2    20.27   
10      2014-07-03 00:02:06           4.1           306    20.27   
11      2014-07-03 00:02:21           3.9         298.6    20.23   
12      2014-07-03 00:02:36           3.6         294.6    20.23   
13      2014-07-03 00:02:51           3.3       